In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from data_utils import get_target

In [37]:
DATA_DIR_PATH = './../data/'

In [38]:
DATASET_NAME = 'students'

In [39]:
target = 'grade'

In [42]:
data = pd.read_csv(f'{DATA_DIR_PATH}{DATASET_NAME}.csv', sep=';')
data = data.dropna(axis=0)
data


In [6]:
target_col = data[target]
if target_col.dtype not in ['float64', 'int64']:
    label_ecnoder = LabelEncoder()
    target_col = label_ecnoder.fit_transform(target_col)
data = data.drop(target, axis=1)
data = data.select_dtypes(include=['float64', 'int64'])
feat_cols = data.columns
data['target'] = target_col

In [7]:
from CrossingBuilder import build_crossings, build_crossings_prototypes
from crossing_functions import crossing_product
from scipy.stats import pointbiserialr as pcorr

In [8]:
depth = 1

In [9]:
prototypes = build_crossings_prototypes(data.drop([target], axis=1), depth)

In [10]:
from importances import get_point_biserial_corrs

In [11]:
orig_imp = get_point_biserial_corrs(data, target)

In [12]:
crossed_data, cross_info = build_crossings(data, prototypes, crossing_product, orig_imp)

In [13]:
crossed_imp = abs(crossed_data.corr()[target]).sort_values()

In [14]:
pd.DataFrame(crossed_imp)

,target
restecg_X_sex,0.019229
fbs_X_thalach,0.019388
fbs_X_sex,0.031167
fbs_X_slope,0.038021
fbs_X_restecg,0.038749
...,...
exang_X_trestbps,0.445097
cp_X_slope,0.449473
cp_X_thalach,0.454156
oldpeak_X_thal,0.457514


In [15]:
cross_info

{'age_X_age': (0.22932355126761092, ['age', 'age']),
 'age_X_sex': (0.2795007572922631, ['age', 'sex']),
 'age_X_cp': (0.43485425005273654, ['age', 'cp']),
 'age_X_trestbps': (0.22932355126761092, ['age', 'trestbps']),
 'age_X_chol': (0.22932355126761092, ['age', 'chol']),
 'age_X_fbs': (0.22932355126761092, ['age', 'fbs']),
 'age_X_restecg': (0.22932355126761092, ['age', 'restecg']),
 'age_X_thalach': (0.42289549648287084, ['age', 'thalach']),
 'age_X_exang': (0.43802855005584634, ['age', 'exang']),
 'age_X_oldpeak': (0.43844127011194434, ['age', 'oldpeak']),
 'age_X_slope': (0.3455117514359984, ['age', 'slope']),
 'age_X_ca': (0.3820852890386706, ['age', 'ca']),
 'age_X_thal': (0.33783815045718624, ['age', 'thal']),
 'sex_X_sex': (0.2795007572922631, ['sex', 'sex']),
 'cp_X_sex': (0.43485425005273654, ['cp', 'sex']),
 'sex_X_trestbps': (0.2795007572922631, ['sex', 'trestbps']),
 'chol_X_sex': (0.2795007572922631, ['chol', 'sex']),
 'fbs_X_sex': (0.2795007572922631, ['fbs', 'sex']),
 

In [16]:
crossed_full_info = pd.DataFrame(crossed_imp)
crossed_full_info['importance'] = crossed_full_info['target']
crossed_full_info = crossed_full_info.drop('target', axis=1)
crossed_full_info['rank'] = pd.Series(dict({k: v[0] for k, v in cross_info.items()}))
crossed_full_info['originals'] = pd.Series(dict({k: v[1] for k, v in cross_info.items()}))
crossed_full_info = crossed_full_info.drop('target', axis=0)

In [17]:
crossed_full_info

,importance,rank,originals
restecg_X_sex,0.019229,0.279501,"[restecg, sex]"
fbs_X_thalach,0.019388,0.422895,"[fbs, thalach]"
fbs_X_sex,0.031167,0.279501,"[fbs, sex]"
fbs_X_slope,0.038021,0.345512,"[fbs, slope]"
fbs_X_restecg,0.038749,0.134468,"[fbs, restecg]"
...,...,...,...
exang_X_thal,0.444305,0.438029,"[exang, thal]"
exang_X_trestbps,0.445097,0.438029,"[exang, trestbps]"
cp_X_slope,0.449473,0.434854,"[cp, slope]"
cp_X_thalach,0.454156,0.434854,"[cp, thalach]"


In [18]:
crossed_full_info.to_csv(f'../data/output/crossed_imp_{DATASET_NAME}_d{depth}.csv')

In [19]:
pd.DataFrame(orig_imp, columns=['importance']).to_csv(f'../data/output/orig_imp_{DATASET_NAME}.csv', index=True)